# Australian Wildfires Dashboard

## Components of Dashboard and Expected layout

### Components of the Dashboard

1. Select Region
2. Select Year
3. Divison to display
    - Pie Chart to display Monthly Average Estimated Fire Area for the selected Regions in the selected Year
    - Bar Chart to display Monthly Average Count of Pixels for Presumed Vegetation Fires for the selected Regions in the selected Year

### Expected Layout

![Final Dashboard](https://raw.githubusercontent.com/sduzair/python/data_vis_dashboard_wildfire_activities_australia/data_vis_dashboard_wildfire_activities_australia/practice_ass_layout.png)

### Requirements to create the expected result

- A dropdown menu: For choosing year
- A radioitem for choosing the Region
- The layout will be designed as follows:
- An outer division with two inner divisions (as shown in the expected layout)
- One of the inner divisions will have information about the radioitem and dropdown (which are the input) and the other one is for adding graphs(the 2 output graphs).
- Callback function to compute data, create graph and return to the layout.

### To do:

1. Import required libraries and read the dataset
2. Create an application layout
3. Add title to the dashboard using HTML H1 component
4. Add a radioitem using dcc.RaioItems and dropdown using dcc.dropdown
5. Add the pie chart and bar chart core graph components.
6. Run the app

## Dependencies

```bash
cat python_version.txt
Python 3.10.12


In [1]:
!python --version

Python 3.10.12


In [2]:
%pip install -q pandas==2.0.3 plotly==5.15.0 dash==2.16.1 dash-bootstrap-components

## Imports

In [3]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import warnings
%config InlineBackend.figure_format = 'retina'

warnings.filterwarnings('ignore')

## Loading data

In [4]:
# Read the wildfire data into pandas dataframe
df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

## Data wrangling

In [5]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df: pd.DataFrame):
    # Change column type to datetime64[ns] for column: 'Date'
    df = df.astype({'Date': 'datetime64[ns]'})
    # Created column 'Year' from formula
    df['Year'] = df['Date'].dt.year
    # Created column 'Month' from formula
    df['Month'] = df['Date'].dt.month
    # Change column type to category for column: 'Month'
    df = df.astype({'Month': 'category'})
    # Change column type to category for column: 'Replaced'
    df = df.astype({'Replaced': 'category'})
    # Change column type to category for column: 'Region'
    df = df.astype({'Region': 'category'})
    # Change column type to int16 for column: 'Count'
    df = df.astype({'Count': 'int16'})
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced,Year,Month
0,NSW,2005-01-04,8.68000,312.266667,42.400000,78.666667,2.886751,8.333333,3,R,2005,1
1,NSW,2005-01-05,16.61125,322.475000,62.362500,85.500000,8.088793,65.428571,8,R,2005,1
2,NSW,2005-01-06,5.52000,325.266667,38.400000,78.333333,3.214550,10.333333,3,R,2005,1
3,NSW,2005-01-07,6.26400,313.870000,33.800000,92.200000,7.529940,56.700000,5,R,2005,1
4,NSW,2005-01-08,5.40000,337.383333,122.533333,91.000000,7.937254,63.000000,3,R,2005,1


In [6]:
def display_mem_usage(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    mem_usage = pd.DataFrame({'df': df.memory_usage(deep=True), 'df_clean': df_clean.memory_usage(deep=True)})
    total = mem_usage.sum().to_frame().T
    total.index = ['Total']

    return pd.concat([total, mem_usage]).T

display_mem_usage(df, df_clean)

,Total,Count,Date,Estimated_fire_area,Index,Mean_confidence,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Month,Region,Replaced,Std_confidence,Var_confidence,Year
df,6314465.0,211248.0,1741476.0,211248.0,128.0,211248.0,211248.0,211248.0,NaN,1562577.0,1531548.0,211248.0,211248.0,NaN
df_clean,1717740.0,52812.0,211248.0,211248.0,128.0,211248.0,211248.0,211248.0,26690.0,27120.0,26630.0,211248.0,211248.0,105624.0


## Pie Chart to display Monthly Average Estimated Fire Area for the selected Regions in the selected Year

In [7]:
# Pie Chart to display Monthly Average Estimated Fire Area for the selected Regions in the selected Year
month_map = {1:'Jan', 2:'Feb', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}

def avg_efa_by_month_pie(df: pd.DataFrame) -> go.Figure:
  df = df_for_year_reg.groupby('Month')['Estimated_fire_area'].mean().reset_index()
  df['Month label'] = df['Month'].map(month_map)

  return px.pie(
      data_frame=df,
      names='Month label',
      values='Estimated_fire_area',
      title=f'{region}: Monthly Average Estimated Fire Area in the Year {year}')

year = 2005
region = 'NSW'
df_for_year_reg = df_clean[(df_clean['Year'] == year) & (df_clean['Region'] == region)]
avg_efa_by_month_pie(df_for_year_reg)


## Bar Chart to display Monthly Average Count of Pixels for Presumed Vegetation Fires for the selected Regions in the selected Year

In [8]:
# Bar Chart to display Monthly Average Count of Pixels for Presumed Vegetation Fires for the selected Regions in the selected Year
def avg_count_by_month_bar(df: pd.DataFrame) -> go.Figure:
  df = df.groupby('Month')['Count'].mean().reset_index()
  df['Month label'] = df['Month'].map(month_map)

  return px.bar(
      data_frame=df,
      x='Month label',
      y='Count',
      title=f'{region}: Monthly Average Count of Pixels for Presumed Vegetation Fires in the year {year}'
  )

avg_count_by_month_bar(df_for_year_reg)

## Dash App

In [9]:
# Initialize the app - incorporate a Dash Bootstrap theme
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
  dbc.Row([html.Div('Title of Dash App', className="text-primary text-center fs-3")]),
  dbc.Row([
    dbc.Label("Region"),
    dbc.RadioItems(options=[{"label": x, "value": x} for x in df_clean['Region'].unique()], value='NSW', inline=True, id='radio-region')
  ]),
  dbc.Row([
    dbc.Label("Year"),
    dcc.Dropdown(df_clean['Year'].unique(), value='2005', id='dropdown-year'),
  ]),
  dbc.Row([
    dbc.Col([dcc.Graph(figure={}, id='fig-pie')], width=6),
    dbc.Col([dcc.Graph(figure={}, id='fig-bar')], width=6),
  ]),
], fluid=True)

@callback(
  Output('fig-pie', 'figure'),
  Output('fig-bar', 'figure'),
  Input('dropdown-year', 'value'),
  Input('radio-region', 'value')
)
def update_graph(year, region):
  df_for_year_reg = df_clean[(df_clean['Year'] == year) & (df_clean['Region'] == region)]
  df_pie = avg_efa_by_month_pie(df_for_year_reg)
  df_bar = avg_count_by_month_bar(df_for_year_reg)
  return df_pie, df_bar

if __name__ == '__main__':
  app.run(debug=True)

<IPython.core.display.Javascript object>